# Notes

This notebook was originally created and stored on Google Colaboratory. For convenience, output has been saved according to the intended input (stored on Google Drive). This is for display and not intended to be ran locally.

This notebook runs simulates our research project's serverside processes for testing purposes. A review set (source: Yelp via Kaggle.com) is cut down into a 'volunteer set', a training set that will be the source of recommendations for our initial 'test users' (also simulated in another notebook). The 'volunteer set' will be clustered using K-Means Clustering, after which Weighted Non-Negative Matrix Factorization will be performed. Output will consist of an U and V matrices, which can be 'downloaded' by the 'test users'. 

# Testing clusters with WNMF
Here we test our clusters with the WNMF implementation using alternating steps.

In [0]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### Filepath
This is the filepath to the yelp review data set.

In [0]:
fp = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Ian/yelp_detailed_small.csv'

Importing the modules and reading file to begin clustering.

In [0]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

df = pd.read_csv(fp)
df.head(5)

,user_name_id,business_name_id,stars,date
0,Ellie: hLQWU4wXuPihOzqCRmzYHg,"""Beaumont Tire & Auto Repair"": -1TUzO4XCmOeqi9...",5,2014-10-23
1,Sheryl: 4hnBlZWXN7fWoaP1HHNfgA,"""Alexander's Steakhouse"": CpNMXASiwtJv5eCDf0n63g",4,2015-11-03
2,Christi: PVyZXgOkVtnU6966FDFhuw,"""The Cookie Jar"": FTky74MxFIMvAJepeUUzEQ",3,2013-05-04
3,Christi: PVyZXgOkVtnU6966FDFhuw,"""Maize Mexican Grill"": VIJ2KiDKhUVhhpNylEIfog",5,2013-05-01
4,Christi: PVyZXgOkVtnU6966FDFhuw,"""Black Dog Smoke & Ale House"": 9MnbQg7kfb_Wgxo...",5,2013-02-09


We cut down our dataset according to year

In [0]:
# x is the chosen start year, where everything before is cut off
x = 2007                              

df = df.sort_values(by = 'date')
# For each year, exclude rows whose 'date' column contains the string of the year
for i in range(2004, x, 1):
  df = df[~df.date.str.contains(str(i))]
df = df.reset_index(drop = True)
df.head(5)

,user_name_id,business_name_id,stars,date
0,Sarah: gxRw9c2lvijXBNhKU5i7_g,"""Mandarin Wok"": e88VWa9xdHmSFdMZne9eLQ",3,2007-01-03
1,Tammy: 8c53BdBWPcbioN3WZJWdCA,"""The Cake Artist's Studio"": rNbAIbTn0zN8yNk0IV...",4,2007-01-04
2,Tammy: 8c53BdBWPcbioN3WZJWdCA,"""The Y Eatery"": aKiE0aZ6vGyOlH-uVOHONw",4,2007-01-04
3,Tammy: 8c53BdBWPcbioN3WZJWdCA,"""Mirabelle Fine Pastries"": RWd83o4drIOE60W1meaESA",5,2007-01-04
4,Tammy: 8c53BdBWPcbioN3WZJWdCA,"""Bacaro"": jeTfL2kCyBtmFGSrSQHqVw",5,2007-01-05


## Selecting a subset to cluster
Here we select the first 1000 reviews to cluster

In [0]:
# x is the chosen cut off for the amount of reviews to cluster
x = 1000

df = df.iloc[0:x]
len(df)

1000

## Pivot the data
We pivot the table into our high dimensional utility matrix

In [0]:
df = df.pivot(index = 'user_name_id', columns = 'business_name_id', values = 'stars')
df = df.fillna(value = 0)
df.head()

business_name_id,"""A-Ri-Rang"": ZMcbVIEXsLO7j1Q1GXKPSw","""Activities and Recreation Center - ARC"": zAD08AT1GgX-cVNy9iMq4A","""Aldi"": tDTUSKxPUUkpFaxBZZl4Cw","""Alexander's Steakhouse"": CpNMXASiwtJv5eCDf0n63g","""Alexandra's"": rOOsFYRPiTOcOnaiChJklQ","""All Creatures Animal Hospital"": zkieW2F82--bbOIRhOCj0g","""Alto Vineyards"": M8G8S2takaE_NOVutGzEkg","""Am-Ko Oriental Foods & Gifts"": zPPw_vUatLmvnRpW1OoX6w","""Amtrak"": jZeLcFDCfBhAxjq1Owe65A","""Anderson Dental"": N-IwlZQimaUrxYF_jGUCOg","""Animal Emergency Clinic of Champaign County"": BhfQ8Rh4L-ca_so-3QjWoA","""Anita Purves Nature Center"": vV6oNhupyM8gEhjmhqLPjQ","""Antonio's Pizza"": BoTQepQTjGbTXA4g4Hj-WA","""Applebee's Neighborhood Grill & Bar"": r_QcMIgY2zxdcV40kK_rxw","""Armored Gopher Games"": cn999G2xZNMj8ZYDot0C8w","""Aroma Cafe"": 0FrYsoVHheQGoXEQsH2d2Q","""Art Coop"": gZI007iLBe_LZbrHSDxHtg","""Art Mart"": dVBp-ayxdvwG95BYfqKwDw","""Assembly Hall"": V7CbZXLAG_wg-wZfQxCZcA","""Atlanta Bread"": nG59soBP2SaQ25rf2n-0bA","""B Won"": C_Zj7H7bUJ5kGCwFmEwW6w","""BJ Grand Salon & Spa"": ChjKd6HgLKmUAQnsrpDSBA","""Bacaro"": jeTfL2kCyBtmFGSrSQHqVw","""Bacca Cigar"": u68_uFbvsqrwZVeHo-a71g","""Bagelmen's Inc"": xo_53Ec0MacJEC5ux7dWhQ","""Bar Guiliani"": tEkUEOsxj9h0pzClYrGXMQ","""Barfly"": 67STv2NXNvt7K8sdbjqetQ","""Basil Thai Cafe"": XtJj67rKT16a4tQw7bxtyw","""Basil Thai Urbana"": w5Y_FiGPSlPemJVxAZZToQ","""Basmati Indian Cuisine"": 7U7uLS9YjjhBM1SoyNH30w","""Bella Mia Boutique"": 2t8Z3VwvV6YpW2gnP-VHig","""Bentley's Pub"": u4D2VypIx3nMsRRbOwx-sA","""Bevande Coffee"": N5PfEojrY4rFqpqzno4aZg","""Biaggi's Ristorante Italiano"": qeJnMI5RmyJ2TZvkqvjn6Q","""Big Lots - Champaign"": OYpRlyukZZh9zH2mxxv4rg","""Big Mouth's"": Z2W3K8x9cRGXvlGWerZH-w","""Billy Barooz Bar & Grill"": PmVxbit6HDDsEUS-j9aDfg","""Blain's Farm & Fleet"": tLgKerW16F9V239MjNA2PA","""Blues"": Q2bnRzJ8AC-3lWyQY8DZqA","""Bo Bo China"": EVkytEhlC1nswqmgrHdviw",...,"""The Bread Company"": Ah4i15g8Ow_zphzcpulTxQ","""The Cake Artist's Studio"": rNbAIbTn0zN8yNk0IV7dfQ","""The Canopy Club"": SVXpyYPAuvJVKcfZ0nMKyg","""The Estate Sale"": pWxCiLYvlUHd9TVJ7n0mEQ","""The Fubar Lounge"": PWs6xJQJPHxknd3FcdKn9Q","""The Great Impasta"": pH72Y8aqqJlq3bgtj-d3UA","""The Habitat for Humanity ReStore"": fZQ3QjGMRELHG9f0WnN8vw","""The Home Depot"": 6vsjWxIMHs-34L9wuTBJRw","""The New Sweet Indulgence"": nplkC6vnh4qT9xH-vhup6w","""The Pita Pit"": DS3-yphtWDHAdXLtIoqdAA","""The Red Herring Vegetarian Restaurant"": kKCwp86xU9XKRnAALQDhrw","""The Ribeye"": PBmfdx-tC2D54FI3HtcKww","""The Y Eatery"": aKiE0aZ6vGyOlH-uVOHONw","""Thomas M. Siebel Center for Computer Science"": jDlvFXuxis4rC2NcWQbqig","""Timpone's"": 0grgvnq4GgoY-estWytUhg","""Tuesday Morning"": RDHH5aVSsblGfOf17P2Tjg","""UPS"": LQRcuOgluaRRMyTH4i3FPw","""University Group"": Hsvqv6AQQ13SvO0uGFu1zg","""University High School"": AqzK7Dr-9zMoxBpsZNlkoA","""University of Illinois Veterinary Teaching Hospital"": D5quwbQTguD4hqMAEARnBg","""University of Illinois"": ZiLOXVloAOtr9_cpQTpr8g","""Urbana Dog Park"": FM6hRJtjDwNeZWa64wpdRg","""Urbana Free Library"": wmhFEY8IctqfSBO0NDcb1w","""Wal-Mart"": 4VYi3I-nVttZeOmrGEXHjQ","""Walnut Street Tea"": pLBF8QrRkMicJcsjAqW7ag","""Ward & Associates Realtors"": yL0PKzDCONu3jHpJG4tQSQ","""Wienerschnitzel"": QHbrfA9nSNLFcevsfQk6dQ","""Will Am-Fm Radio"": YBNS9AHuSSMSVPGlsgvqbA","""William M Staerkel Planetarium"": DNBKTu7qoN_18JIybyIVPw","""Wingstop"": 4ZAy5ZCTWbduzzhtR0egHQ","""Women's Health Practice"": c_pScQ7eKYTesWkcqDMn-g","""Wonderdogs"": SuQpsHxcxCAB8kxPEIXiBg","""Woori Jib Restaurant"": GWS5s8HSm1zjqMLJD7wQLg","""World Harvest Foods"": LElQBiDw8HyU_LrbjP8C1A","""Yellow Checker Cab"": J6TGHzwKKq5bqBR4_WsnUA","""Yellowfin Japanese Restaurant"": C31ExBTn_6UxbTVkWPtNkg","""Yuko Hair Salon"": 4LXNVWV_Yp8HbkikmRQ2gg","""Za's Italian Cafe"": FUr2uEolARu7rv2TQYKPqg","""Zelma's"": h2vFVV5pJjn8arA7GR6mQg","""Zorba's Restaurant"": kyXEnWKQGWSThY6EcjORuw"
user_name_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Clustering the subset
We use KMeans to cluster the UM and make it into the dataframe.

In [0]:
# x is the chosen number of clusters
x = 25

m = KMeans(n_clusters = x)
m.fit_predict(df)
df_og = df     # This reference to the original was made to record the column names for later use. See section # Exporting the U and V matrix
df = pd.DataFrame(m.cluster_centers_)
df.head(25)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355
0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.500000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.500000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,5.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,3.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
2,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000,0.000000,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000,4.000000e+00,0.000000,4.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,3.000000,0.000000e+00,0.000000
3,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,5.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,5.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,4.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0

## Performing WNMF on the cluster
Here we have the WNMF algorithm

In [0]:
#choose number of iterations
iterations = 4

#choose number of latent factors
latent_factors = 10

#choose limit of convergence
limit = 0.2

#####################################################

um = df.to_numpy()

#convert NaN values to 0
a = np.nan_to_num(um)

#create matrix w of weights: 1 for observed values, else 0
w = np.zeros((len(a), len(a[0])), dtype=int)
for i in range(len(a)):
    for j in range(len(a[i])):
        if a[i][j] != 0:
            w[i][j] = 1

#u/v matrix initialization with random values
u = np.random.rand(len(a), latent_factors)
v = np.random.rand(latent_factors, len(a[0]))

iteration = 0
prev_norm = 0
curr_norm = 0
change = 999999
while(iteration < iterations and change > limit):
  u_it = np.nditer(u, flags=['multi_index'])
  v_it = np.nditer(v, flags=['multi_index'])
  while (not u_it.finished or not v_it.finished):
    if not u_it.finished:
      i, j = u_it.multi_index
      num = np.matmul((w[i,:] * a[i,:]), v.T[:,j])
      denom = np.matmul(w[i,:] * np.matmul(u[i,:], v), v.T[:,j])
      u[i][j] = u_it[0] * (num / denom)
      u_it.iternext()
    if not v_it.finished:
      i, j = v_it.multi_index
      num = np.matmul(u.T[i,:], (w[:,j] * a[:,j]))
      denom = np.matmul(u.T[i,:], (w[:,j] * np.matmul(u, v[:,j])))
      v[i][j] = v_it[0] * (num / denom)
      v_it.iternext()

  prev_norm = curr_norm
  curr_norm = np.linalg.norm((np.multiply(w, (a - np.matmul(u, v)))), ord='fro')
  change = abs(curr_norm - prev_norm)
  print('Iteration: ' + str(iteration) + ' Previous Norm: ' + str(prev_norm) + ' Current Norm: ' + str(curr_norm) + ' Change: ' + str(change))
  iteration += 1

print('Final WNMF-produced prediction matrix:')
uv = np.matmul(u, v)
uv = pd.DataFrame(uv)
uv.head(25)

Iteration: 0 Previous Norm: 0 Current Norm: 19.822645904025325 Change: 19.822645904025325
Iteration: 1 Previous Norm: 19.822645904025325 Current Norm: 14.582806466933036 Change: 5.239839437092289
Iteration: 2 Previous Norm: 14.582806466933036 Current Norm: 12.979507826507144 Change: 1.6032986404258924
Iteration: 3 Previous Norm: 12.979507826507144 Current Norm: 11.971327811322194 Change: 1.0081800151849496
Final WNMF-produced prediction matrix:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355
0,2.294704,1.884448,4.018878,2.904688,3.364096,0.174305,2.677568,2.468012,1.207079,6.176017,0.253859,3.001816,2.274146,0.123966,2.334174,2.476703,3.312782,3.468712,1.754194,2.254708,3.457784,2.239822,2.867792,3.092155,2.571552,1.735118,1.975966,1.079675,2.077663,1.754102,2.544210,2.987775,0.563255,2.555350,2.959804,2.301162,2.788607,2.914118,2.569185,0.652913,...,2.097952,2.441955,6.836549,3.212877,2.294696,2.859520,2.863573,2.819103,4.879440,3.073752,2.805105,3.133756,2.026434,2.502847,2.132380,1.596620,0.518072,1.875896,3.872080,2.642837,2.840895,3.142000,3.459729,0.622584,3.010978,2.649415,1.525640,2.256928,2.957146,3.652305,3.106288,2.910111,2.780015,3.287653,1.422522,2.728118,3.657587,2.093192,1.888576,3.276700
1,4.056540,2.206521,4.842915,3.766455,4.272970,0.138873,4.437384,3.480799,1.559394,8.754292,0.219276,4.394182,3.945686,0.111581,4.733448,2.826996,4.864820,3.012650,3.084062,3.108592,3.732284,2.594275,3.429815,4.794514,3.241333,3.494555,2.943418,1.500705,2.502054,1.890269,3.665834,4.653373,0.642900,3.092438,4.184376,2.245933,3.382490,3.722736,4.778342,0.803477,...,4.318827,3.895486,10.709537,4.194197,3.161294,3.460462,3.998046,4.265083,5.093929,3.741062,3.399409,4.020477,3.240147,4.254002,3.224426,2.093167,1.059506,2.697086,3.517774,3.700738,4.274311,4.594388,3.891654,0.786015,4.569575,4.307694,2.311494,2.811223,4.121055,5.803893,4.316784,3.747819,3.367554,3.830535,2.471366,3.859544,6.476388,3.458509,1.859222,3.811795
2,2.968283,2.227526,4.422814,3.262489,4.032526,0.686305,4.018469,3.179055,2.028442,6.561207,0.618750,4.258376,3.448345,0.285860,3.646066,2.741358,4.235926,4.262242,3.034287,2.972812,3.462868,4.328149,4.375213,4.411923,3.613151,3.486239,2.920230,1.545867,2.205549,2.210140,2.950386,3.818288,0.684235,2.786048,3.113689,2.410434,4.100123,4.063405,4.282312,1.005433,...,2.363650,4.040665,11.581939,4.032292,3.905779,3.573061,3.964091,3.727534,4.374133,3.237840,2.386653,3.973724,3.559095,4.668711,3.131583,1.962387,1.016188,2.935371,4.707445,4.102642,3.190211,3.915969,4.784625,0.778378,4.119550,3.943059,2.522927,3.468849,4.451082,3.792743,3.808338,3.342368,4.325958,5.625682,1.908041,3.520978,6.200536,3.042093,1.926701,5.071067
3,3.183409,2.202678,5.607835,3.864881,4.858348,0.156933,4.339538,3.958701,1.647200,7.900369,0.234712,5.043395,3.550821,0.114610,3.768579,3.527370,5.050671,4.961579,3.015294,3.684164,5.273573,3.067711,4.471288,5.002366,3.384583,2.588088,2.456931,1.576182,3.222636,2.611807,4.203201,4.663078,0.668002,3.716092,4.304288,2.853266,3.941078,4.485901,3.390084,1.134875,...,3.190023,4.003613,10.702128,4.970217,3.652621,3.526618,4.752157,3.938035,7.914307,4.034791,4.028204,3.677724,3.474625,4.116324,3.192144,2.339139,0.838671,2.448879,5.311423,4.192413,4.547090,4.378660,5.218149,0.814761,4.159919,4.740930,2.732944,3.454401,4.642200,5.010272,5.108917,5.113017,3.365786,4.660044,2.584509,4.236115,7.208960,2.637046,2.687336,4.393442
4,0.030529,0.018889,0.042554,0.027809,0.030559,0.004245,0.025045,0.023676,0.015878,0.033988,0.005640,0.032447,0.024226,0.002742,0.024287,0.024463,0.030718,0.033244,0.021611,0.025282,0.021706,0.034654,0.032194,0.032076,0.030613,0.021468,0.018071,0.013521,0.013295,0.016882,0.021246,0.027935,0.007050,0.015955,0.022988,0.024384,0.024736,0.034092,0.033858,0.008090,...,0.021585,0.032994,0.075519,0.020924,0.026492,0.025047,0.031252,0.029738,0.024090,0.026581,0.020629,0.033835,0.026009,0.037438,0.026467,0.011612,0.007719,0.024110,0.034905,0.030644,0.024059,0.031750,0.031368,0.007112,0.032494,0.021124,0.015547,0.027688,0.032293,0.034634,0.027248,0.018550,0.033545,0.041784,0.015044,0.026697,0.034594,0.028187,0.019016,0.044707
5,3.540114,2.703354,5.836581,4.161698,5.309736,0.103869,4.741169,4.134426,1.718300,11.298746

## Exporting the U and V matrix
Renaming the columns so they match the businesses and then exporting for use on the user-side.

In [0]:
df_v = pd.DataFrame(v)
df_v.columns = df_og.columns

df_u = pd.DataFrame(u)

df_v.head()

business_name_id,"""A-Ri-Rang"": ZMcbVIEXsLO7j1Q1GXKPSw","""Activities and Recreation Center - ARC"": zAD08AT1GgX-cVNy9iMq4A","""Aldi"": tDTUSKxPUUkpFaxBZZl4Cw","""Alexander's Steakhouse"": CpNMXASiwtJv5eCDf0n63g","""Alexandra's"": rOOsFYRPiTOcOnaiChJklQ","""All Creatures Animal Hospital"": zkieW2F82--bbOIRhOCj0g","""Alto Vineyards"": M8G8S2takaE_NOVutGzEkg","""Am-Ko Oriental Foods & Gifts"": zPPw_vUatLmvnRpW1OoX6w","""Amtrak"": jZeLcFDCfBhAxjq1Owe65A","""Anderson Dental"": N-IwlZQimaUrxYF_jGUCOg","""Animal Emergency Clinic of Champaign County"": BhfQ8Rh4L-ca_so-3QjWoA","""Anita Purves Nature Center"": vV6oNhupyM8gEhjmhqLPjQ","""Antonio's Pizza"": BoTQepQTjGbTXA4g4Hj-WA","""Applebee's Neighborhood Grill & Bar"": r_QcMIgY2zxdcV40kK_rxw","""Armored Gopher Games"": cn999G2xZNMj8ZYDot0C8w","""Aroma Cafe"": 0FrYsoVHheQGoXEQsH2d2Q","""Art Coop"": gZI007iLBe_LZbrHSDxHtg","""Art Mart"": dVBp-ayxdvwG95BYfqKwDw","""Assembly Hall"": V7CbZXLAG_wg-wZfQxCZcA","""Atlanta Bread"": nG59soBP2SaQ25rf2n-0bA","""B Won"": C_Zj7H7bUJ5kGCwFmEwW6w","""BJ Grand Salon & Spa"": ChjKd6HgLKmUAQnsrpDSBA","""Bacaro"": jeTfL2kCyBtmFGSrSQHqVw","""Bacca Cigar"": u68_uFbvsqrwZVeHo-a71g","""Bagelmen's Inc"": xo_53Ec0MacJEC5ux7dWhQ","""Bar Guiliani"": tEkUEOsxj9h0pzClYrGXMQ","""Barfly"": 67STv2NXNvt7K8sdbjqetQ","""Basil Thai Cafe"": XtJj67rKT16a4tQw7bxtyw","""Basil Thai Urbana"": w5Y_FiGPSlPemJVxAZZToQ","""Basmati Indian Cuisine"": 7U7uLS9YjjhBM1SoyNH30w","""Bella Mia Boutique"": 2t8Z3VwvV6YpW2gnP-VHig","""Bentley's Pub"": u4D2VypIx3nMsRRbOwx-sA","""Bevande Coffee"": N5PfEojrY4rFqpqzno4aZg","""Biaggi's Ristorante Italiano"": qeJnMI5RmyJ2TZvkqvjn6Q","""Big Lots - Champaign"": OYpRlyukZZh9zH2mxxv4rg","""Big Mouth's"": Z2W3K8x9cRGXvlGWerZH-w","""Billy Barooz Bar & Grill"": PmVxbit6HDDsEUS-j9aDfg","""Blain's Farm & Fleet"": tLgKerW16F9V239MjNA2PA","""Blues"": Q2bnRzJ8AC-3lWyQY8DZqA","""Bo Bo China"": EVkytEhlC1nswqmgrHdviw",...,"""The Bread Company"": Ah4i15g8Ow_zphzcpulTxQ","""The Cake Artist's Studio"": rNbAIbTn0zN8yNk0IV7dfQ","""The Canopy Club"": SVXpyYPAuvJVKcfZ0nMKyg","""The Estate Sale"": pWxCiLYvlUHd9TVJ7n0mEQ","""The Fubar Lounge"": PWs6xJQJPHxknd3FcdKn9Q","""The Great Impasta"": pH72Y8aqqJlq3bgtj-d3UA","""The Habitat for Humanity ReStore"": fZQ3QjGMRELHG9f0WnN8vw","""The Home Depot"": 6vsjWxIMHs-34L9wuTBJRw","""The New Sweet Indulgence"": nplkC6vnh4qT9xH-vhup6w","""The Pita Pit"": DS3-yphtWDHAdXLtIoqdAA","""The Red Herring Vegetarian Restaurant"": kKCwp86xU9XKRnAALQDhrw","""The Ribeye"": PBmfdx-tC2D54FI3HtcKww","""The Y Eatery"": aKiE0aZ6vGyOlH-uVOHONw","""Thomas M. Siebel Center for Computer Science"": jDlvFXuxis4rC2NcWQbqig","""Timpone's"": 0grgvnq4GgoY-estWytUhg","""Tuesday Morning"": RDHH5aVSsblGfOf17P2Tjg","""UPS"": LQRcuOgluaRRMyTH4i3FPw","""University Group"": Hsvqv6AQQ13SvO0uGFu1zg","""University High School"": AqzK7Dr-9zMoxBpsZNlkoA","""University of Illinois Veterinary Teaching Hospital"": D5quwbQTguD4hqMAEARnBg","""University of Illinois"": ZiLOXVloAOtr9_cpQTpr8g","""Urbana Dog Park"": FM6hRJtjDwNeZWa64wpdRg","""Urbana Free Library"": wmhFEY8IctqfSBO0NDcb1w","""Wal-Mart"": 4VYi3I-nVttZeOmrGEXHjQ","""Walnut Street Tea"": pLBF8QrRkMicJcsjAqW7ag","""Ward & Associates Realtors"": yL0PKzDCONu3jHpJG4tQSQ","""Wienerschnitzel"": QHbrfA9nSNLFcevsfQk6dQ","""Will Am-Fm Radio"": YBNS9AHuSSMSVPGlsgvqbA","""William M Staerkel Planetarium"": DNBKTu7qoN_18JIybyIVPw","""Wingstop"": 4ZAy5ZCTWbduzzhtR0egHQ","""Women's Health Practice"": c_pScQ7eKYTesWkcqDMn-g","""Wonderdogs"": SuQpsHxcxCAB8kxPEIXiBg","""Woori Jib Restaurant"": GWS5s8HSm1zjqMLJD7wQLg","""World Harvest Foods"": LElQBiDw8HyU_LrbjP8C1A","""Yellow Checker Cab"": J6TGHzwKKq5bqBR4_WsnUA","""Yellowfin Japanese Restaurant"": C31ExBTn_6UxbTVkWPtNkg","""Yuko Hair Salon"": 4LXNVWV_Yp8HbkikmRQ2gg","""Za's Italian Cafe"": FUr2uEolARu7rv2TQYKPqg","""Zelma's"": h2vFVV5pJjn8arA7GR6mQg","""Zorba's Restaurant"": kyXEnWKQGWSThY6EcjORuw"
0,0.589713,0.189171,0.282108,0.008324,0.924193,6.457445e-06,0.934173,0.765077,0.151

In [0]:
filepath = '/content/gdrive/My Drive/Summer Research 2019/Individual Work Folder/Minh/'

df_u.to_csv((filepath + 'u.csv'))
df_v.to_csv((filepath + 'v.csv'))